# V1.0 RAG with Custom Prompt
This version focus on RAG with custom prompt.\
Custom prompt is designed to invoke code generation by the LLM.

In [13]:
%pip install --user -qU langchain
%pip install --user -qU langchain_community
%pip install --user -qU langchain_chroma
%pip install --user -qU langchain-openai
%pip install --user -qU langchainhub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [15]:
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [16]:
folder_path = "ca-md"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = UnstructuredMarkdownLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [18]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())

In [19]:
retriever = vectorstore.as_retriever()

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

## Prompt 1


In [21]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [25]:
response = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response["answer"])

To create a strategy based on the Relative Strength Index (RSI) for buying Bitcoin when its RSI drops below 30 and selling when it goes above 70, you can use the following Python code snippet as a basic example:

```python
# Let's assume you have a function to calculate RSI for Bitcoin
def calculate_rsi(data):
    # Calculate RSI logic here
    # This function should return the RSI value
    return rsi_value

# Assuming you have a function to buy Bitcoin
def buy_bitcoin():
    print("Buying Bitcoin")

# Assuming you have a function to sell Bitcoin
def sell_bitcoin():
    print("Selling Bitcoin")

# Let's assume you have historical RSI data for Bitcoin
bitcoin_rsi = [35, 25, 20, 65, 75, 80, 40, 30, 35, 70]

# Implementing the strategy
for rsi in bitcoin_rsi:
    if rsi < 30:
        buy_bitcoin()
    elif rsi > 70:
        sell_bitcoin()
```

In this code snippet:
- The `calculate_rsi` function represents the logic to calculate the RSI value for Bitcoin based on historical data.
- The `

In [27]:
for document in response["context"]:
    print(f"Source: {document.metadata['source']}")
    #print('#############################################################')

Source: rsi.md
Source: rsi.md
Source: rsi.md
Source: rsi.md


LLM is able to find the correct documentation of the technical indicator (in this case RSI). \
However, the retriever is only able to retrieve documentation for the technical indicator and not the necessary markdown documentations for generating strategy code (hello-world.mdx).